In [1]:
import pandas as pd
import numpy as np

In [3]:
trip = pd.read_csv('data/tripP')
weather = pd.read_csv('data/weatherP')
station = pd.read_csv('data/stationP')

In [4]:
trip = trip.merge(station, on='start_station_id', how='left')

In [5]:
trip = trip.merge(weather, on=['date', 'zip_code'], how='left')
#SOLUCION RATA
trip.fillna(0, inplace=True)
trip.drop(['date', 'zip_code'], axis=1, inplace=True)

In [6]:
trip = trip.drop(['holiday'], axis=1)

In [8]:
train = pd.read_csv('data/trip_train.csv', usecols=['id'])
train = train.merge(trip, on='id', how='left')

In [9]:
test = pd.read_csv('data/trip_test.csv', usecols=['id'])
test = test.merge(trip, how="left", on="id")

In [39]:
from sklearn import ensemble
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
r = ensemble.ExtraTreesRegressor(n_jobs=-1)
param_dist = {"max_depth": [None, 1, 2, 3, 4, 5],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False]}

In [40]:
cols = ['start_station_id', 'subscription_type', 'time', 'year', 'month', 'day',
'dayofweek', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f', 'max_humidity',
'mean_humidity', 'min_humidity', 'max_sea_level_pressure_inches',
'mean_sea_level_pressure_inches', 'min_sea_level_pressure_inches',
'max_visibility_miles', 'mean_visibility_miles', 'min_visibility_miles',
'max_wind_Speed_mph', 'mean_wind_speed_mph', 'max_gust_speed_mph',
'precipitation_inches', 'cloud_cover', 'events', 'wind_dir_degrees']

random_search = RandomizedSearchCV(r, param_distributions=param_dist, n_iter=20)
random_search.fit(train[cols], train.duration)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
          verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9259b67e10>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9264aaa350>, 'bootstrap': [True, False], 'max_depth': [None, 1, 2, 3, 4, 5], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9264aaa090>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=0)

In [42]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.017 (std: 0.012)
Parameters: {'max_features': 6, 'min_samples_split': 5, 'bootstrap': False, 'max_depth': 2, 'min_samples_leaf': 5}

Model with rank: 2
Mean validation score: 0.016 (std: 0.012)
Parameters: {'max_features': 9, 'min_samples_split': 3, 'bootstrap': False, 'max_depth': 1, 'min_samples_leaf': 5}

Model with rank: 3
Mean validation score: 0.014 (std: 0.010)
Parameters: {'max_features': 10, 'min_samples_split': 5, 'bootstrap': True, 'max_depth': 2, 'min_samples_leaf': 4}



In [45]:
from sklearn.metrics import mean_squared_error
mean_squared_error(
    ensemble.ExtraTreesRegressor(
        n_jobs=-1,
        max_features=10,
        min_samples_split=5,
        bootstrap=True,
        max_depth=2,
        min_samples_leaf=4)\
            .fit(train[cols], train.duration)\
            .predict(test[cols]),
    test.duration)

41829582.730453528